# Clinical Fairness Analysis Demo

This notebook demonstrates the Obermeyer kidney algorithm bias case and how to detect and remediate it using the Clinical Fairness Intervention System.

## Background

The Obermeyer et al. (2019) study revealed significant racial bias in a widely-used healthcare algorithm for chronic kidney disease management. The algorithm systematically underestimated disease severity for Black patients, leading to reduced access to specialist care.

**Key findings:**
- The algorithm used healthcare costs as a proxy for health needs
- Black patients had lower healthcare costs due to reduced access to care
- This created a feedback loop where bias in the system reinforced bias in the algorithm

## Objectives

1. Reproduce the bias pattern using synthetic clinical data
2. Use causal analysis to identify bias pathways
3. Detect fairness violations using multiple metrics
4. Generate and apply fairness interventions
5. Evaluate the effectiveness of interventions

## Setup

In [ ]:
import sys
import os
from pathlib import Path

# Add src directory to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Import our modules
from causal_analysis import CausalAnalyzer
from bias_detection import BiasDetector
from intervention_engine import InterventionEngine
from code_generator import CodeGenerator

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Setup complete!")

## 1. Load and Explore Data

In [ ]:
# Load synthetic clinical data
data_path = Path.cwd().parent / "data" / "sample" / "demo_data.csv"
data = pd.read_csv(data_path)

print(f"Loaded {len(data)} patient records")
print(f"\nColumns: {list(data.columns)}")
print(f"\nData shape: {data.shape}")

data.head(10)

In [ ]:
# Examine data distribution
print("\n=== Data Summary ===")
print(f"\nRace distribution:")
print(data['race'].value_counts())

print(f"\nGender distribution:")
print(data['gender'].value_counts())

print(f"\nReferral rate: {data['referral'].mean():.2%}")

print(f"\nReferral rate by race:")
print(data.groupby('race')['referral'].agg(['mean', 'count']))

In [ ]:
# Visualize outcome distribution by protected attribute
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Referral rate by race
data.groupby('race')['referral'].mean().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Referral Rate by Race', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Referral Rate')
axes[0].set_xlabel('Race')
axes[0].axhline(y=data['referral'].mean(), color='red', linestyle='--', label='Overall Mean')
axes[0].legend()

# Creatinine levels by race and referral
data.boxplot(column='creatinine_level', by=['race', 'referral'], ax=axes[1])
axes[1].set_title('Creatinine Levels by Race and Referral Status', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Race, Referral')
axes[1].set_ylabel('Creatinine Level')

plt.tight_layout()
plt.show()

print("\nInitial observation: Disparate referral rates across racial groups")

## 2. Train a Biased Model (Baseline)

In [ ]:
# Encode categorical variables
data_encoded = data.copy()
le_race = LabelEncoder()
le_gender = LabelEncoder()
le_insurance = LabelEncoder()

data_encoded['race_encoded'] = le_race.fit_transform(data['race'])
data_encoded['gender_encoded'] = le_gender.fit_transform(data['gender'])
data_encoded['insurance_encoded'] = le_insurance.fit_transform(data['insurance_type'])

print("Race encoding:", dict(enumerate(le_race.classes_)))

# Prepare features
feature_cols = [
    'age', 'race_encoded', 'gender_encoded', 'creatinine_level',
    'chronic_conditions', 'insurance_encoded', 'prior_visits',
    'distance_to_hospital'
]

X = data_encoded[feature_cols + ['race']]  # Keep race for fairness analysis
y = data_encoded['referral']

# Train-test split (stratified by race)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=data_encoded['race']
)

# Train baseline model
baseline_model = LogisticRegression(random_state=42, max_iter=1000)
baseline_model.fit(X_train[feature_cols], y_train)

# Evaluate
y_pred_baseline = baseline_model.predict(X_test[feature_cols])
accuracy = accuracy_score(y_test, y_pred_baseline)

print(f"\nBaseline Model Accuracy: {accuracy:.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': baseline_model.coef_[0]
}).sort_values('coefficient', ascending=False)

print("\nFeature Importance (Coefficients):")
print(feature_importance)

## 3. Causal Analysis - Identify Bias Pathways

In [ ]:
# Initialize causal analyzer
analyzer = CausalAnalyzer(data, 'race', 'referral')

# Infer causal graph
graph, explanation = analyzer.infer_causal_graph(llm_enhanced=False)

print("\n=== CAUSAL GRAPH ANALYSIS ===")
print(explanation)

# Identify bias pathways
pathways = analyzer.identify_bias_pathways()

print("\n=== BIAS PATHWAYS ===")
print(f"Found {len(pathways)} causal pathways from race to referral:\n")
for i, path in enumerate(pathways, 1):
    print(f"{i}. {' -> '.join(path)}")

## 4. Bias Detection - Quantify Fairness Violations

In [ ]:
# Initialize bias detector
detector = BiasDetector(['race'])

# Compute fairness metrics
metrics = detector.compute_fairness_metrics(
    baseline_model, X_test, y_test, 'race'
)

# Generate report
report = detector.generate_bias_report(metrics, 'race')
print(report)

In [ ]:
# Visualize fairness metrics
group_metrics = metrics['group_metrics']
groups = list(group_metrics.keys())

# Prepare data for visualization
metrics_df = pd.DataFrame({
    'Group': groups,
    'Positive Rate': [group_metrics[g]['positive_rate'] for g in groups],
    'TPR (Recall)': [group_metrics[g]['true_positive_rate'] for g in groups],
    'FPR': [group_metrics[g]['false_positive_rate'] for g in groups],
    'Accuracy': [group_metrics[g]['accuracy'] for g in groups]
})

# Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Positive Rate
axes[0, 0].bar(metrics_df['Group'], metrics_df['Positive Rate'], color='steelblue')
axes[0, 0].set_title('Positive Prediction Rate by Group', fontweight='bold')
axes[0, 0].set_ylabel('Rate')
axes[0, 0].axhline(y=metrics_df['Positive Rate'].mean(), color='red', linestyle='--')

# TPR vs FPR
x = np.arange(len(groups))
width = 0.35
axes[0, 1].bar(x - width/2, metrics_df['TPR (Recall)'], width, label='TPR', color='green')
axes[0, 1].bar(x + width/2, metrics_df['FPR'], width, label='FPR', color='red')
axes[0, 1].set_title('TPR vs FPR by Group', fontweight='bold')
axes[0, 1].set_ylabel('Rate')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(groups)
axes[0, 1].legend()

# Accuracy
axes[1, 0].bar(metrics_df['Group'], metrics_df['Accuracy'], color='purple')
axes[1, 0].set_title('Accuracy by Group', fontweight='bold')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].axhline(y=metrics_df['Accuracy'].mean(), color='red', linestyle='--')

# Fairness criteria summary
fairness_data = {
    'Metric': ['Demographic\nParity', 'Equalized\nOdds', 'Equal\nOpportunity'],
    'Difference': [
        metrics['demographic_parity']['difference'],
        metrics['equalized_odds']['average_difference'],
        metrics['equal_opportunity']['difference']
    ]
}
colors = ['red' if d > 0.1 else 'green' for d in fairness_data['Difference']]
axes[1, 1].bar(fairness_data['Metric'], fairness_data['Difference'], color=colors)
axes[1, 1].set_title('Fairness Criteria Violations', fontweight='bold')
axes[1, 1].set_ylabel('Difference')
axes[1, 1].axhline(y=0.1, color='orange', linestyle='--', label='Threshold (0.1)')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 5. Intervention Recommendations

In [ ]:
# Get intervention recommendations
engine = InterventionEngine(bias_threshold=0.1)
recommendations = engine.suggest_interventions(
    metrics,
    max_recommendations=5,
    prioritize_accuracy=True
)

print("\n=== INTERVENTION RECOMMENDATIONS ===")
print(f"\nGenerated {len(recommendations)} recommendations:\n")

for rec in recommendations:
    print(f"\n{rec.priority}. {rec.name} ({rec.category})")
    print(f"   Description: {rec.description}")
    print(f"   Expected Impact: {rec.expected_impact}")
    print(f"   Complexity: {rec.complexity}")
    print(f"   Preserves Accuracy: {rec.preserves_accuracy}")
    print(f"   Parameters: {rec.parameters}")

## 6. Generate Intervention Code

In [ ]:
# Generate code for top recommendation
if recommendations:
    top_intervention = recommendations[0]
    
    print(f"\nGenerating code for: {top_intervention.name}")
    
    generator = CodeGenerator()
    code_result = generator.generate_fix_code(top_intervention.name)
    
    print("\n=== GENERATED CODE ===")
    print("\nImports:")
    for imp in code_result.imports:
        print(f"  {imp}")
    
    print("\nCode:")
    print(code_result.code[:1000] + "..." if len(code_result.code) > 1000 else code_result.code)
    
    print("\nUsage Example:")
    print(code_result.usage_example[:500] + "..." if len(code_result.usage_example) > 500 else code_result.usage_example)

## 7. Key Takeaways

### Findings:
1. **Causal Pathways**: Identified direct and indirect bias pathways from race to referral decisions
2. **Fairness Violations**: Detected violations in demographic parity, equalized odds, and equal opportunity
3. **Intervention Strategy**: Generated actionable recommendations to mitigate bias

### Recommendations:
- Apply preprocessing interventions (reweighing/resampling) to balance training data
- Consider postprocessing methods to adjust predictions for fairness
- Monitor fairness metrics continuously in production
- Combine technical interventions with policy changes

### Next Steps:
1. Implement the recommended intervention
2. Re-evaluate fairness metrics
3. Assess fairness-accuracy tradeoffs
4. Deploy with ongoing monitoring

### References:
- Obermeyer, Z., et al. (2019). "Dissecting racial bias in an algorithm used to manage the health of populations." Science, 366(6464), 447-453.
- AIF360: IBM's AI Fairness 360 Toolkit
- Fairlearn: Microsoft's Fairness Assessment and Improvement Toolkit

In [ ]:
# Save results for further analysis
print("\nNotebook execution complete!")
print("\nTo continue:")
print("1. Implement the recommended intervention")
print("2. Use the generated code as a starting point")
print("3. Re-run bias detection to validate improvements")